In [1]:
import pandas as pd
import numpy as np

In [2]:
departments=pd.read_csv("/content/MOCK_DATA1.csv")
projects=pd.read_csv("/content/MOCK_DATA2.csv")
employees=pd.read_csv("/content/MOCK_DATA.csv")

In [3]:
departments=departments['name']

In [4]:
departments.drop_duplicates(inplace=True)

In [5]:
departments = departments.reset_index(drop=True)

In [6]:
departments = departments.reset_index().rename(columns={'index': 'id'})

In [7]:
departments['id'] = departments['id'] + 1

In [8]:
departments.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12 entries, 0 to 11
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      12 non-null     int64 
 1   name    12 non-null     object
dtypes: int64(1), object(1)
memory usage: 324.0+ bytes


In [9]:
departments["name"]=departments["name"].convert_dtypes(pd.StringDtype)

In [10]:
departments.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12 entries, 0 to 11
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      12 non-null     int64 
 1   name    12 non-null     string
dtypes: int64(1), string(1)
memory usage: 324.0 bytes


In [11]:
departments.to_csv("departments.csv",index=False)

In [12]:
projects = projects[["name", "start_date", "end_date"]]

In [13]:
projects.drop_duplicates(subset="name",inplace=True)

<ipython-input-13-89e348fecd6d>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  projects.drop_duplicates(subset="name",inplace=True)


In [14]:
projects = projects.reset_index(drop=True)

In [15]:
projects = projects.reset_index().rename(columns={'index': 'id'})

In [16]:
projects['id'] = projects['id'] + 1

In [17]:
projects.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39 entries, 0 to 38
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   id          39 non-null     int64 
 1   name        39 non-null     object
 2   start_date  39 non-null     object
 3   end_date    39 non-null     object
dtypes: int64(1), object(3)
memory usage: 1.3+ KB


In [18]:
projects["start_date"] = pd.to_datetime(projects["start_date"])

In [19]:
projects["end_date"] = pd.to_datetime(projects["end_date"])

In [20]:
projects["name"] = projects["name"].convert_dtypes(pd.StringDtype)

In [21]:
projects.to_csv("projects.csv",index=False)

In [22]:
employees.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   id             1000 non-null   int64 
 1   name           1000 non-null   object
 2   department_id  1000 non-null   object
 3   salary         1000 non-null   object
 4   hire_date      1000 non-null   object
dtypes: int64(1), object(4)
memory usage: 39.2+ KB


In [23]:
employees["name"] = employees["name"].convert_dtypes(pd.StringDtype)

In [24]:
employees["salary"] = employees["salary"].convert_dtypes(pd.Int64Dtype)

In [25]:
employees["hire_date"] = pd.to_datetime(employees["hire_date"])

In [26]:
employees.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   id             1000 non-null   int64         
 1   name           1000 non-null   string        
 2   department_id  1000 non-null   object        
 3   salary         1000 non-null   string        
 4   hire_date      1000 non-null   datetime64[ns]
dtypes: datetime64[ns](1), int64(1), object(1), string(2)
memory usage: 39.2+ KB


In [27]:
employees=pd.merge(employees,departments,how="inner",left_on="department_id",right_on="name")

In [28]:
employees=employees[["id_x","name_x","id_y","salary","hire_date"]]

In [29]:
employees=employees.rename(columns={"id_x":"id","name_x":"name","id_y":"department_id"})

In [30]:
employees.to_csv("employees.csv",index=False)

In [31]:
projects=projects.head(20)

In [32]:
employees.columns

Index(['id', 'name', 'department_id', 'salary', 'hire_date'], dtype='object')

In [33]:
projects.columns

Index(['id', 'name', 'start_date', 'end_date'], dtype='object')

In [34]:
# Add a sequential index to both DataFrames
employees['emp_order'] = range(len(employees))
projects['proj_order'] = range(len(projects))

<ipython-input-34-8df17ab9bbb8>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  projects['proj_order'] = range(len(projects))


In [35]:
# Perform the cross-join
merged_df = pd.merge(employees, projects, how='cross')

In [36]:
# Function to randomly select N projects for each employee
def select_random_projects(group, n=2):
    if len(group) >= n:
        return group.sample(n=n)
    else:
        return group # Or handle cases with fewer projects differently

In [37]:
# Group by employee ID and apply the selection function
assigned_multiple_df = merged_df.groupby('id_x', group_keys=False).apply(select_random_projects).reset_index(drop=True)

<ipython-input-37-bf69b6cd9db8>:2: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  assigned_multiple_df = merged_df.groupby('id_x', group_keys=False).apply(select_random_projects).reset_index(drop=True)


In [38]:
# Rename columns as needed - PROVIDE 11 NAMES
assigned_multiple_df.columns = ['employee_id', 'employee_name', 'department_id', 'salary', 'hire_date', 'employee_order',
                                  'project_id', 'project_name', 'project_start_date', 'project_end_date', 'project_order']

In [39]:
print(assigned_multiple_df.head())
print(f"Number of assigned rows: {len(assigned_multiple_df)}")

   employee_id employee_name  department_id     salary  hire_date  \
0            1  Maren Lowing              2  $89276.92 2023-01-06   
1            1  Maren Lowing              2  $89276.92 2023-01-06   
2            2  Doria Dawley              7  $10965.53 2018-01-27   
3            2  Doria Dawley              7  $10965.53 2018-01-27   
4            3  Selie Jimmes             11  $94600.13 2015-08-06   

   employee_order  project_id project_name project_start_date  \
0               0          12     Treeflex         2024-10-26   
1               0          14       Namfix         2024-08-23   
2               1          12     Treeflex         2024-10-26   
3               1           9        Otcom         2024-06-19   
4               2          12     Treeflex         2024-10-26   

  project_end_date  project_order  
0       2024-04-04             11  
1       2025-02-05             13  
2       2024-04-04             11  
3       2025-03-29              8  
4       2024-0

In [40]:
# Create the 'assigned_date' column using the 'project_start_date'
assigned_multiple_df['assigned_date'] = assigned_multiple_df['project_start_date']

# Display the first few rows with the new column
print(assigned_multiple_df.head())

# Verify the new column is present in the DataFrame's columns
print(assigned_multiple_df.columns)

   employee_id employee_name  department_id     salary  hire_date  \
0            1  Maren Lowing              2  $89276.92 2023-01-06   
1            1  Maren Lowing              2  $89276.92 2023-01-06   
2            2  Doria Dawley              7  $10965.53 2018-01-27   
3            2  Doria Dawley              7  $10965.53 2018-01-27   
4            3  Selie Jimmes             11  $94600.13 2015-08-06   

   employee_order  project_id project_name project_start_date  \
0               0          12     Treeflex         2024-10-26   
1               0          14       Namfix         2024-08-23   
2               1          12     Treeflex         2024-10-26   
3               1           9        Otcom         2024-06-19   
4               2          12     Treeflex         2024-10-26   

  project_end_date  project_order assigned_date  
0       2024-04-04             11    2024-10-26  
1       2025-02-05             13    2024-08-23  
2       2024-04-04             11    2024-10

employee_projects (
  employee_id INT,
  project_id INT,
  assigned_date DATE,
  PRIMARY KEY (employee_id, project_id)
)

In [41]:
assigned_multiple_df.columns

Index(['employee_id', 'employee_name', 'department_id', 'salary', 'hire_date',
       'employee_order', 'project_id', 'project_name', 'project_start_date',
       'project_end_date', 'project_order', 'assigned_date'],
      dtype='object')

In [42]:
assigned_multiple_df=assigned_multiple_df[['employee_id','project_id','assigned_date']]

In [43]:
assigned_multiple_df.to_csv("employee_projects",index=False)

In [45]:
assigned_multiple_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   employee_id    2000 non-null   int64         
 1   project_id     2000 non-null   int64         
 2   assigned_date  2000 non-null   datetime64[ns]
dtypes: datetime64[ns](1), int64(2)
memory usage: 47.0 KB
